In [4]:
import gm_analysis as gm

# Quality control

In [2]:
!docker run -it nipreps/mriqc:latest --version


MRIQC v22.0.6


**Participant level**

In [9]:
!docker run -it --rm -v {gm.info.data_paths.bids_dir}:/data:ro -v {gm.info.data_paths.qc_dir}:/out nipreps/mriqc:latest /data /out participant --participant_label sub-00 sub-0000


221011-10:49:31,858 cli IMPORTANT:
	 
    Running MRIQC version 22.0.6:
      * BIDS dataset path: /data.
      * Output folder: /out.
      * Analysis levels: ['participant'].

221011-10:49:46,949 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will be submitted to MRIQC's metrics repository. Submission of IQMs can be disabled using the ``--no-sub`` argument. Please visit https://mriqc.readthedocs.io/en/latest/dsa.html to revise MRIQC's Data Sharing Agreement.
221011-12:49:28,421 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:29,885 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:29,902 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:29,902 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:29,911 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:31,887 cli WARNING:
	 Building bold report: no exclude index was found
221011-12:49:31,90

**Group level**

In [5]:
!docker run -it --rm -v {gm.info.data_paths.bids_dir}:/data:ro -v {gm.info.data_paths.qc_dir}:/out nipreps/mriqc:latest /data /out group


# fmriprep

In [2]:
print(gm.info.data_paths.root_dir)
print(gm.info.data_paths.bids_dir)

print(gm.info.data_paths.derivatives_dir)
print(gm.info.data_paths.scratch_dir)
print(gm.info.data_paths.fmriprep_dir)


/media/marius/data_ex/gaze-motion
/media/marius/data_ex/gaze-motion/dsgm
/media/marius/data_ex/gaze-motion/derivatives
/media/marius/data_ex/gaze-motion/derivatives/scratch
/media/marius/data_ex/gaze-motion/derivatives/fmriprep


+ incorporate pre-run freesurfer: https://neurostars.org/t/fmriprep-how-to-incorporate-pre-run-freesurfer/1425

In [3]:
!fmriprep-docker --version

/opt/conda/lib/python3.8/site-packages/bids/config.py:39: FutureWarning: Setting 'extension_initial_dot' will be removed in pybids 0.16.
  warnings.warn("Setting 'extension_initial_dot' will be removed in pybids 0.16.",
You are using fMRIPrep-21.0.0, and a newer version of fMRIPrep is available: 21.0.1.
Please check out our documentation about how and when to upgrade:
https://fmriprep.readthedocs.io/en/latest/faq.html#upgrading
fMRIPrep v21.0.0


In [ ]:
!fmriprep-docker {gm.info.data_paths.bids_dir} {gm.info.data_paths.fmriprep_dir} participant --participant_label 0000 -w {gm.info.data_paths.scratch_dir}

----

In [ ]:
!nipypecli crash /out/fmriprep/sub-09/log/20191128-091018_1542d3ed-6675-4c4b-9236-6271b743dd40/crash-20191128-101315-root-autorecon2_vol-868eb950-4da0-4a67-bce7-603d93342d66.txt

# Clean images

In [13]:
import sys
import importlib
from os.path import join as opj
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from nilearn import image
from nilearn.masking import apply_mask
from nilearn.input_data import NiftiMasker, NiftiSpheresMasker
from nilearn.interfaces import fmriprep



# import gm_analysis
import gm_analysis as gma



sns.set_style('white')
sns.set_context('notebook')
palette = sns.color_palette('Set1')

%matplotlib inline

In [3]:
# init new dict if not existing
if 'subjects' not in locals():
        subjects = {}

        
sub_ses_list = {
    'sub-00': ['ses-gc', 'ses-com'], 
    'sub-0000':['ses-com', 'ses-sep']
}

for sub_id in sub_ses_list:
    subjects[sub_id] = {}
    if not sub_ses_list[sub_id] == '':
        for ses_ in sub_ses_list[sub_id]:
            subjects[sub_id][ses_] = gma.subject.Subject(sub_id, ses_, gma.info.data_paths)
    else:
        subjects[sub_id] = gma.subject.Subject(sub_id, sub_ses_list[sub_id], gma.info.data_paths)


In [10]:
subjects["sub-0000"]['ses-sep'].info.func.go['go_run-1'].img

'/media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-go_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

In [11]:
img_ = image.load_img(subjects["sub-0000"]['ses-sep'].info.func.go['go_run-1'].img)

In [12]:
np.shape(img_)

(65, 77, 65, 261)

In [19]:
confs, sample_mask = fmriprep.load_confounds(
    subjects["sub-0000"]['ses-sep'].info.func.go['go_run-1'].img,
    strategy=['motion', 'high_pass', 'wm_csf', 'compcor', 'scrub'],
    motion='full',
    scrub=0,
    fd_threshold=.5,
    std_dvars_threshold=3,
    wm_csf='basic',
    global_signal='basic',
    compcor='anat_combined',
    n_compcor=2,
    demean=True)

In [20]:
sample_mask

array([  0,   1,   2,   3,   4,   5,   6,   9,  16,  17,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
       125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137,
       138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 18

In [21]:
img_indexed = image.index_img(img_, sample_mask)

In [22]:
np.shape(img_indexed)

(65, 77, 65, 248)

In [4]:
for standardize in [False, True]:
    for fwhm in [None, 5]:
        subjects["sub-0000"]['ses-sep'].clean_images(standardize=standardize, fwhm=fwhm)

Image stored:  /media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-faceLoc_run-1_space-MNI152NLin2009cAsym_desc-preprocCleaned_bold.nii.gz stored
Image stored:  /media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-faceLoc_run-2_space-MNI152NLin2009cAsym_desc-preprocCleaned_bold.nii.gz stored
Image stored:  /media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-faceLoc_run-3_space-MNI152NLin2009cAsym_desc-preprocCleaned_bold.nii.gz stored
Image stored:  /media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-faceLoc_run-4_space-MNI152NLin2009cAsym_desc-preprocCleaned_bold.nii.gz stored
Image stored:  /media/marius/data_ex/gaze-motion/derivatives/fmriprep/sub-0000/ses-sep/func/sub-0000_ses-sep_task-faceLoc_run-5_space-MNI152NLin2009cAsym_desc-preprocCleaned_bold.nii.gz stored
Image stored:  /media/marius/data_e